<a href="https://colab.research.google.com/github/Annmodels/mnist_recon/blob/master/DnRGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#%pylab notebook
import cv2
import tensorflow as tf
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import zipfile
import time

In [0]:
from google.colab import files
files.upload()

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c cifar-10

In [0]:
os.chdir('/content')
!pwd
path = 'final'
os.makedirs(path)
os.chdir(path)
!pwd
!cp '/content/train.7z' '/content/final/'
!ls
!7z e train.7z


In [0]:
!pwd

!rm '/content/final/train.7z'
os.chdir('/content/final/')

!rmdir train
img_db = []
base_path = '/content/'
cartoon = '/content/final'
for img in os.listdir(cartoon):
  try:
    img_ary = cv2.imread(os.path.join(cartoon,img))
    img_db.append(img_ary)
  except Exception as e:
      pass
img_db = np.array(img_db)
print(img_db.shape)

In [0]:
def plot_images(img):
  fig , axes = plt.subplots(2 , 8 , figsize = (13,3))
  for i in range(8):
    axes[0,i].imshow(img[i])
    axes[0,i].axis('off')
    axes[1,i].imshow(img[i+8])
    axes[1,i].axis('off')
  #fig.show()
  #plt.draw()
  #plt.close()

In [0]:

img_db = np.array(img_db)
print(img_db.shape)
total_imgs = 2000
img_db = np.array(img_db[0:total_imgs])
original = img_db.astype('float32')
original = original/255
print(original.shape)

plot_images(original)
                     

In [0]:
noisy = original + 0.1*np.random.normal(loc = 0.0,scale = 1.0,size = original.shape)
original = np.clip(original,0,1)
plot_images(noisy) #.astype(uint8))
xt = np.transpose(original,(0,3,1,2))
xt_noise = np.transpose(noisy,(0,3,1,2))
print(xt.shape)
print(xt_noise.shape)
xtflat = xt.reshape(-1,1024)
xtflat_noise = xt_noise.reshape(-1,1024)
print(xtflat.shape)
print(xtflat_noise.shape)
                                

In [0]:
lrate = 0.001
epochs = 10000
batchsz = 100
displaystep = 100
ex2show = 8

In [0]:
hidden1 = 512
hidden2 = 256
hidden3 = 128
ninput = 1024

In [0]:
x = tf.placeholder('float32',[None,ninput])
y = tf.placeholder('float32',[None,ninput])


In [0]:
weights = {'enh1':tf.Variable(tf.truncated_normal([ninput,hidden1],stddev = 0.01)),
           'enh2':tf.Variable(tf.truncated_normal([hidden1,hidden2],stddev = 0.01)),
           'enh3':tf.Variable(tf.truncated_normal([hidden2,hidden3],stddev = 0.01)),
           'deh1':tf.Variable(tf.truncated_normal([hidden3,hidden2],stddev = 0.01)),
           'deh2':tf.Variable(tf.truncated_normal([hidden2,hidden1],stddev = 0.01)),
           'deh3':tf.Variable(tf.truncated_normal([hidden1,ninput],stddev = 0.01))
          }
biases = {'enb1':tf.Variable(tf.truncated_normal([hidden1],stddev = 0.01)),
          'enb2':tf.Variable(tf.truncated_normal([hidden2],stddev = 0.01)),
          'enb3':tf.Variable(tf.truncated_normal([hidden3],stddev = 0.01)),
          'deb1':tf.Variable(tf.truncated_normal([hidden2],stddev = 0.01)),
          'deb2':tf.Variable(tf.truncated_normal([hidden1],stddev = 0.01)),
          'deb3':tf.Variable(tf.truncated_normal([ninput],stddev = 0.01))
}


In [0]:
def encoder(x):
  layer1 = tf.nn.sigmoid(tf.add(tf.matmul(x,weights['enh1']),biases['enb1']))
  layer2 = tf.nn.sigmoid(tf.add(tf.matmul(layer1,weights['enh2']),biases['enb2']))
  layer3 = tf.nn.sigmoid(tf.add(tf.matmul(layer2,weights['enh3']),biases['enb3']))
  return layer3
def decoder(x):
  layer1 = tf.nn.sigmoid(tf.add(tf.matmul(x,weights['deh1']),biases['deb1']))
  layer2 = tf.nn.sigmoid(tf.add(tf.matmul(layer1,weights['deh2']),biases['deb2']))
  layer3 = tf.nn.sigmoid(tf.add(tf.matmul(layer2,weights['deh3']),biases['deb3']))
  return layer3

In [0]:
encoder_op = encoder(x)
decoder_op = decoder(encoder_op)
ypred = decoder_op
cost = tf.reduce_mean(tf.pow(y-ypred,2))
optimizer = tf.train.RMSPropOptimizer(lrate).minimize(cost)
init = tf.global_variables_initializer()

In [0]:
starttime = time.time()
totalbatch = int(xtflat.shape[0]/100) #batchsz)
sess = tf.Session()
sess.run(init)
for epoch in range(epochs):
  start = 0
  end = batchsz
  for i in range(totalbatch-1):
    index = np.arange(start,end)
    np.random.shuffle(index)
    batchxs = xtflat[index]
    batchxsn = xtflat_noise[index]
    start = end
    end = start + batchsz
    _,c = sess.run([optimizer,cost],feed_dict = {x:batchxsn,y:batchxs})
  if (epoch%100 == 0):
    print('epoch:  {0:05d}  loss:{1:f}'.format(epoch,c))
print("finished")
endtime = time.time()
print('time tkn:{0}'.format(endtime-starttime))

In [0]:
indext = np.random.randint(original.shape[0],size = ex2show)
print(indext)
indext = np.sort(indext)
print(indext)
rgbdx = np.concatenate([indext*3,indext*3+1,indext*3+2])
print(rgbdx)
rgbdx = np.sort(rgbdx)
print(rgbdx)
denoised = sess.run(ypred,feed_dict = {x:xtflat_noise[rgbdx]})
print(denoised.shape)
denoised = np.reshape(denoised,(ex2show,3,32,32))
print(denoised.shape)
denoised = np.transpose(denoised,(0,2,3,1))
print(denoised.shape)

In [0]:
f , a = plt.subplots(3,ex2show,figsize = (13,5))
for i in range(ex2show):
  a[0][i].imshow(original[indext[i]])
  a[0,i].axis('off')
  a[1][i].imshow(noisy[indext[i]])
  a[1,i].axis('off')
  a[2][i].imshow(denoised[i])
  a[2,i].axis('off')
f.show()
plt.draw()


In [0]:
def plot_max_active(x):
  fig,axes = plt.subplots(10 , 10, figsize = (13,10))
  fig.subplots_adjust(hspace = .1 , wspace = 0)
  h = w = np.sqrt(x.shape[0]).astype(int)

  for i,ax in enumerate(axes.flat):
    ax.imshow(x[:,i].reshape((h,w)))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
  plt.show()
plot_max_active(sess.run(weights['enh3']))


In [0]:
plt.close('all')